In [25]:
import pandas as pd
import numpy as np
from scipy.stats import stats

from datetime import datetime
from datetime import timedelta

In [6]:
import os
import pandas as pd

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Задание 1

Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

In [7]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_users = read_database('experiment_users.csv')

In [29]:
df1 = (
    df_sales
    [(df_sales['date'] >= datetime(2022, 3, 16)) & (df_sales['date'] < datetime(2022, 3, 23))]
    .groupby('user_id', as_index=False)['price'].sum()
)

df_exp = df_users.merge(df1, how='left', on='user_id').fillna(0)

data_control = df_exp[df_exp['pilot'] == 0]['price']
data_test = df_exp[df_exp['pilot'] == 1]['price']

np.round(stats.ttest_ind(data_control, data_test).pvalue, 3)

/var/folders/1p/ybd4mcmj71j_w9gphd0xdrgc6vsdmk/T/ipykernel_1715/171184530.py:12: DeprecationWarning: Please use `ttest_ind` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  np.round(stats.ttest_ind(data_control, data_test).pvalue, 3)


0.199

# Задание 2

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

In [34]:
user_sales = df_sales.groupby('user_id', as_index=False)['sale_id'].nunique()
user_with_sales = user_sales[user_sales['sale_id'] > 1]['user_id'].unique()

df2 = df_sales[df_sales['user_id'].isin(user_with_sales)]
df2 = df2.sort_values(['user_id', 'date'])

df2['prev_date'] = df2.groupby('user_id')['date'].shift(1)

(df2['date'] - df2['prev_date']).dt.days.mean().round()